# Explorative Data Analysis


### Import necessary packages

In [1]:
from match import Match
from utils import save_to_json

### Exploring the data

In [2]:
match_1_instance = Match("../data/match_1.json")
match_2_instance = Match("../data/match_2.json")

In [3]:
match_1_instance.info()
match_2_instance.info()

Number of gaits in this match is: 574
Number of gaits in this match is: 613


## Label

In [4]:
match_1_instance.count_actions()

Counter({'walk': 245,
         'run': 209,
         'dribble': 42,
         'rest': 25,
         'pass': 24,
         'tackle': 20,
         'shot': 6,
         'cross': 3})

In [5]:
match_2_instance.count_actions()

Counter({'run': 343,
         'walk': 190,
         'dribble': 36,
         'shot': 12,
         'tackle': 11,
         'rest': 10,
         'pass': 8,
         'no action': 2,
         'cross': 1})

In [6]:
match_1_instance.extract_sequences()

[['walk', 'walk', 'walk', 'walk', 'rest'],
 ['rest', 'walk'],
 ['walk', 'walk', 'walk', 'run'],
 ['run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'walk'],
 ['walk', 'walk', 'walk', 'walk', 'tackle'],
 ['tackle', 'run'],
 ['run', 'run', 'run', 'run', 'run', 'run', 'run', 'walk'],
 ['walk', 'walk', 'walk', 'run'],
 ['run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'walk'],
 ['walk', 'run'],
 ['run', 'run', 'run', 'run', 'run', 'walk'],
 ['walk', 'run'],
 ['run', 'run', 'tackle'],
 ['tackle', 'dribble'],
 ['dribble', 'run'],
 ['run', 'dribble'],
 ['dribble', 'pass'],
 ['pass', 'pass', 'walk'],
 ['walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'run'],
 ['run', 'run', 'run', 'walk'],
 ['walk', 'run'],
 ['run', 'walk'],
 ['walk', 'walk', 'run'],
 ['run', 'run', 'rest'],
 ['res

In [7]:
match_2_instance.extract_sequences()

[['no action', 'run'],
 ['run', 'pass'],
 ['pass', 'rest'],
 ['rest', 'walk'],
 ['walk', 'run'],
 ['run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'run',
  'walk'],
 ['walk', 'walk', 'walk', 'run'],
 ['run', 'walk'],
 ['walk', 'walk', 'run'],
 ['run', 'run', 'run', 'run', 'walk'],
 ['walk', 'walk', 'walk', 'walk', 'walk', 'dribble'],
 ['dribble', 'dribble', 'dribble', 'shot'],
 ['shot', 'dribble'],
 ['dribble', 'shot'],
 ['shot', 'walk'],
 ['walk', 'walk', 'tackle'],
 ['tackle', 'dribble'],
 ['dribble', 'dribble', 'cross'],
 ['cross', 'run'],
 ['run', 'run', 'run', 'run', 'walk'],
 ['walk', 'dribble'],
 ['dribble', 'pass'],
 ['pass', 'dribble'],
 ['dribble', 'pass'],
 ['pass', 'walk'],
 ['walk', 'rest'],
 ['rest', 'run'],
 ['run', 'run', 'run', 'run', 'run', 'walk'],
 ['walk', 'walk', 'walk', 'run'],
 ['run', 'run', 'tackle'],
 ['tackle', 'run'],
 ['run', 'tackle'],
 ['tackle', 'run'],
 ['ru

## Norm

In [8]:
accelerometer_dimension = len(match_1_instance.data[0]["norm"])
print(f"Norm dimension: {accelerometer_dimension}")

Norm dimension: 72


In [9]:
# Consistency check
all(len(gait["norm"]) == accelerometer_dimension for gait in match_1_instance.data)

False

In [10]:
match_1_instance.find_min_max()

{'MIN': 1, 'MAX': 227}

In [11]:
match_2_instance.find_min_max()

{'MIN': 3, 'MAX': 723}

In [12]:
match_1_data_with_averaged_norm=match_1_instance.average_data_norm()
print(match_1_data_with_averaged_norm)

[{'label': 'walk', 'norm': 24.166957780185083}, {'label': 'walk', 'norm': 26.026276390692598}, {'label': 'walk', 'norm': 23.998176545905707}, {'label': 'walk', 'norm': 22.59657290497246}, {'label': 'rest', 'norm': 20.378778196399168}, {'label': 'walk', 'norm': 28.23765592340506}, {'label': 'walk', 'norm': 26.56198941145795}, {'label': 'walk', 'norm': 25.081731147723094}, {'label': 'run', 'norm': 30.745144284498725}, {'label': 'run', 'norm': 43.36040697515091}, {'label': 'run', 'norm': 74.81118010786089}, {'label': 'run', 'norm': 47.74688658278297}, {'label': 'run', 'norm': 61.22481878874269}, {'label': 'run', 'norm': 46.98641222326422}, {'label': 'run', 'norm': 41.23867057187769}, {'label': 'run', 'norm': 43.0510509740288}, {'label': 'run', 'norm': 40.85855066950079}, {'label': 'run', 'norm': 34.33202180620319}, {'label': 'run', 'norm': 35.14701851084251}, {'label': 'run', 'norm': 27.330254507744716}, {'label': 'run', 'norm': 39.46746113300068}, {'label': 'run', 'norm': 41.233051351141

In [13]:
match_2_data_with_averaged_norm=match_1_instance.average_data_norm()
print(match_2_data_with_averaged_norm)

[{'label': 'walk', 'norm': 24.166957780185083}, {'label': 'walk', 'norm': 26.026276390692598}, {'label': 'walk', 'norm': 23.998176545905707}, {'label': 'walk', 'norm': 22.59657290497246}, {'label': 'rest', 'norm': 20.378778196399168}, {'label': 'walk', 'norm': 28.23765592340506}, {'label': 'walk', 'norm': 26.56198941145795}, {'label': 'walk', 'norm': 25.081731147723094}, {'label': 'run', 'norm': 30.745144284498725}, {'label': 'run', 'norm': 43.36040697515091}, {'label': 'run', 'norm': 74.81118010786089}, {'label': 'run', 'norm': 47.74688658278297}, {'label': 'run', 'norm': 61.22481878874269}, {'label': 'run', 'norm': 46.98641222326422}, {'label': 'run', 'norm': 41.23867057187769}, {'label': 'run', 'norm': 43.0510509740288}, {'label': 'run', 'norm': 40.85855066950079}, {'label': 'run', 'norm': 34.33202180620319}, {'label': 'run', 'norm': 35.14701851084251}, {'label': 'run', 'norm': 27.330254507744716}, {'label': 'run', 'norm': 39.46746113300068}, {'label': 'run', 'norm': 41.233051351141

In [14]:
match_1_data_with_averaged_norm_instance = Match("../data/match_1_data_with_averaged_norm.json")
match_2_data_with_averaged_norm_instance = Match("../data/match_2_data_with_averaged_norm.json")

In [15]:
match_1_data_with_averaged_norm_instance.count_actions()

Counter({'walk': 245,
         'run': 209,
         'dribble': 42,
         'rest': 25,
         'pass': 24,
         'tackle': 20,
         'shot': 6,
         'cross': 3})

In [16]:
match_2_data_with_averaged_norm_instance.count_actions()

Counter({'run': 343,
         'walk': 190,
         'dribble': 36,
         'shot': 12,
         'tackle': 11,
         'rest': 10,
         'pass': 8,
         'no action': 2,
         'cross': 1})

In [17]:
save_to_json(object_to_save = match_1_instance.average_data_norm(),path="../data/match_1_data_with_averaged_norm.json")

In [18]:
save_to_json(object_to_save = match_2_instance.average_data_norm(),path="../data/match_2_data_with_averaged_norm.json")

In [22]:
mean_norm_for_each_action_dict = {
    action: match_1_data_with_averaged_norm_instance.mean_norm_for_each_action(action)
    for action in list((match_1_data_with_averaged_norm_instance.count_actions()).keys())
}

mean_norm_for_each_action_dict

{'walk': 28.09486687642306,
 'rest': 21.83356684290428,
 'run': 54.40459298904966,
 'tackle': 70.66858172485196,
 'dribble': 52.88252588755935,
 'pass': 64.13327808012248,
 'cross': 46.708457756829354,
 'shot': 113.27871688343295}

In [24]:
mean_norm_for_each_action_dict = {
    action: match_2_data_with_averaged_norm_instance.mean_norm_for_each_action(action)
    for action in list((match_2_data_with_averaged_norm_instance.count_actions()).keys())
}

mean_norm_for_each_action_dict

{'no action': 38.65222535277646,
 'run': 56.22519161734227,
 'pass': 73.00031950031808,
 'rest': 22.712307380205683,
 'walk': 29.869326616095396,
 'dribble': 65.56369756735364,
 'shot': 98.46478534250633,
 'tackle': 66.86037467805848,
 'cross': 93.89217051873887}